In [ ]:
from mandelbrot import mandelbrot

In [ ]:
#%autosave 0
%matplotlib tk
# widget , tk , qt5
# 'module://ipympl.backend_nbagg' , 'TkAgg', 'Qt5Agg'
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'all':lambda x: f"{x:.2E}"})
backend = plt.get_backend()
print('Current backend is %s'%backend)

In [ ]:
plt.close()

In [ ]:
def button_press(event):
    global c, coords, pendingDraw #, left, right, bottom, top
    #plt.ioff()
    print(event.name, c, sep=',', end='; ')
    ax = event.inaxes 
    if event.button == 1:
        #print('# deal with zoom in')
        left, right, bottom, top, rx, ry = coords[c]
        c+=1
        arm_x=(right-left)/4
        arm_y=(top-bottom)/4
        right  = event.xdata + arm_x
        left   = event.xdata - arm_x
        bottom = event.ydata - arm_y
        top    = event.ydata + arm_y
        coords[c] = [left, right, bottom, top, rx, ry ]
        if backend == 'module://ipympl.backend_nbagg':
            ax.set_xlim(left,right)
            ax.set_ylim(bottom,top)
            mandraw(ax, left, right, bottom, top, rx, ry )
        elif backend == 'TkAgg':
            ax._set_view_from_bbox( ax.transData.transform( [ (left,bottom) , (right,top) ]).flatten() )
            event.canvas.draw()
            mandraw(ax, left, right, bottom, top, rx, ry )
        elif backend == 'Qt5Agg':
            ax._set_view_from_bbox( ax.transData.transform( [ (left,bottom) , (right,top) ]).flatten() )
            event.canvas.draw()
            mandraw(ax, left, right, bottom, top, rx, ry )
        ft.set_text('z.In l{},r{},b{},t{}'.format(left, right, bottom, top))
    elif event.button == 3:
        #print('# deal with zoom out')
        if c==0:
            print('Already initial image, not zooming out')
            return
        c-=1
        left, right, bottom, top, rx, ry = coords[c]
        if backend == 'module://ipympl.backend_nbagg':
            ax.set_xlim(left,right)
            ax.set_ylim(bottom,top)
        else:
            ax._set_view_from_bbox( ax.transData.transform( [ (left,bottom) , (right,top) ]).flatten() )
            event.canvas.draw_idle()
            event.canvas.flush_events()
        ft.set_text('z.Out l{},r{},b{},t{}'.format(left, right, bottom, top))
        pendingDraw=False
    else:
        # deal with something that should never happen
        print('wtf!')
    #plt.ion()
    #print('done!')

def resize(event):
    global coords, pendingDraw
    print(event.name, c, sep=',', end='; ')
    rx, ry = get_ax_wh(fig, ax)
    ft.set_text('Resize w{}, h{}, dpi{}'.format( rx, ry, fig.dpi))
    coords[c][4]=rx
    coords[c][5]=ry
    pendingDraw=True
    if backend == 'module://ipympl.backend_nbagg':
        event.canvas.draw_idle()
        event.canvas.flush_events()
    if backend =='TkAgg':
        event.canvas.draw_idle()
        event.canvas.flush_events()
    if backend =='Qt5Agg':
        event.canvas.draw_idle()
        event.canvas.flush_events()

def draw(event):
    global pendingDraw
    if pendingDraw:
        print(event.name, pendingDraw, c, sep=',', end='; ')
        mandraw(ax, *coords[c] )
        if backend == 'module://ipympl.backend_nbagg':
            event.canvas.draw_idle()
            event.canvas.flush_events()
        if backend =='TkAgg':
            event.canvas.draw_idle()
            event.canvas.flush_events()
        if backend =='Qt5Agg':
            event.canvas.draw_idle()
            event.canvas.flush_events()

def get_ax_wh(fig,ax):
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    width, height = int(bbox.width*fig.dpi), int(bbox.height*fig.dpi)    
    return width, height

def mandraw(ax, left, right, bottom, top, rx, ry ):
    global pendingDraw
    dx = (right - left ) / rx
    dy = (top - bottom) / ry
    z = np.zeros((ry,rx), dtype=np.int32)
    mandelbrot( z, left, dx, rx, bottom, dy, ry, 63)
    ax.pcolorfast( (left,right), (bottom,top), z, cmap='terrain', vmin=0, vmax=63)
    pendingDraw=False
    print('mandraw', c, sep=',', end='; ')
    
# init plot
fig, ax = plt.subplots()

fig.canvas.toolbar_position = 'top'
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
ax.set_navigate(True)
#fig.tight_layout()
    
# connect events
ids=dict()
#ids[]=fig.canvas.mpl_connect('',)
#ids['axes_enter']=fig.canvas.mpl_connect('axes_enter_event',axes_enter)
ids['draw'] = fig.canvas.mpl_connect('draw_event', draw) 
ids['resize'] = fig.canvas.mpl_connect('resize_event', resize)
ids['button_press'] = fig.canvas.mpl_connect('button_press_event', button_press)

# Initial data
left, right = np.float64((-2.0, 0.66))
bottom, top = np.float64((-1.4, 1.4))
rx, ry = 30, 40
coords=dict()
c=0
coords[c] = [left, right, bottom, top, rx, ry ] #print(c,coords[c])
pendingDraw=False

# Initial plot
ft = fig.text(0.5,0.9,'w{}, h{}, dpi{}'.format( *get_ax_wh(fig,ax), fig.dpi))
mandraw(ax, left, right, bottom, top, rx, ry )

In [ ]:
backend